##### Copyright 2018 The TensorFlow Authors.



In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Image captioning with visual attention

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/text/image_captioning">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/text/image_captioning.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/image_captioning.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/text/image_captioning.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Given an image like the example below, our goal is to generate a caption such as "a surfer riding on a wave".

![Man Surfing](https://tensorflow.org/images/surf.jpg)

*[Image Source](https://commons.wikimedia.org/wiki/Surfing#/media/File:Surfing_in_Hawaii.jpg); License: Public Domain*

To accomplish this, you'll use an attention-based model, which enables us to see what parts of the image the model focuses on as it generates a caption.

![Prediction](https://tensorflow.org/images/imcap_prediction.png)

The model architecture is similar to [Show, Attend and Tell: Neural Image Caption Generation with Visual Attention](https://arxiv.org/abs/1502.03044).

This notebook is an end-to-end example. When you run the notebook, it downloads the [MS-COCO](http://cocodataset.org/#home) dataset, preprocesses and caches a subset of images using Inception V3, trains an encoder-decoder model, and generates captions on new images using the trained model.

In this example, you will train a model on a relatively small amount of data—the first 30,000 captions  for about 20,000 images (because there are multiple captions per image in the dataset).

In [0]:
#Connect to Drive
from google.colab import drive
drive.mount('/content/drive')

In [0]:
ROOT_DIR = '/content/drive/My Drive/_AI/Image Captioning'

DATASET_PATH = ROOT_DIR + '/dataset'

COCO_dataset = ROOT_DIR + '/coco/'

COCO_dataset_2 = ROOT_DIR + '/coco_2/'

#Path of Zip files
DATASET_ZIP = DATASET_PATH + '/Flickr8k_Dataset.zip'
TEXT_ZIP = DATASET_PATH + '/Flickr8k_text.zip'
GLOVE_ZIP = DATASET_PATH + '/glove.6B.200d.txt.zip'

dataset_img_path = DATASET_PATH +'/Flicker8k_Dataset'
dataset_text_path = DATASET_PATH +'/TextFiles'

# TRAIN_PATH = ROOT_DIR + 'input_1/training_set/'
# SMALL_TRAIN_PATH = ROOT_DIR + 'dataset/train'

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle

## Download and prepare the MS-COCO dataset

You will use the [MS-COCO dataset](http://cocodataset.org/#home) to train our model. The dataset contains over 82,000 images, each of which has at least 5 different caption annotations. The code below downloads and extracts the dataset automatically.

**Caution: large download ahead**. You'll use the training set, which is a 13GB file.

In [0]:
annotation_zip = tf.keras.utils.get_file('captions.zip',
                                          cache_subdir=os.path.abspath('.'),
                                          origin = 'http://images.cocodataset.org/annotations/annotations_trainval2014.zip',
                                          extract = True)
annotation_file = os.path.dirname(annotation_zip)+'/annotations/captions_train2014.json'

# name_of_zip = 'train2014.zip'
# if not os.path.exists(os.path.abspath('.') + '/' + name_of_zip):
#   image_zip = tf.keras.utils.get_file(name_of_zip,
#                                       cache_subdir=os.path.abspath('.'),
#                                       origin = 'http://images.cocodataset.org/zips/train2014.zip',
#                                       extract = True)
#   PATH = os.path.dirname(image_zip)+'/train2014/'
# else:
#   PATH = os.path.abspath('.')+'/train2014/'

In [0]:
name_of_zip = 'train2014.zip'
if not os.path.exists(os.path.abspath('.') + '/' + name_of_zip):
  image_zip = tf.keras.utils.get_file(name_of_zip,
                                      cache_subdir=os.path.abspath('.'),
                                      origin = 'http://images.cocodataset.org/zips/train2014.zip',
                                      extract = True)
  PATH = os.path.dirname(image_zip)+'/train2014/'
else:
  PATH = os.path.abspath('.')+'/train2014/'

## Optional: limit the size of the training set 
To speed up training for this tutorial, you'll use a subset of 30,000 captions and their corresponding images to train our model. Choosing to use more data would result in improved captioning quality.

In [0]:
# COCO Image Caption
annotation{
  "id": int,
  "image_id": int, 
  "caption": str,
  }

In [0]:
# Read the json file
with open(annotation_file, 'r') as f:
    annotations = json.load(f)

# Store captions and image names in vectors
all_captions = []
all_img_name_vector = []

for annot in annotations['annotations']:
    caption = '<start> ' + annot['caption'] + ' <end>'
    image_id = annot['image_id']
    full_coco_image_path = PATH + 'COCO_train2014_' + '%012d.jpg' % (image_id)

    all_img_name_vector.append(full_coco_image_path)
    all_captions.append(caption)

# Shuffle captions and image_names together
# Set a random state
train_captions, img_name_vector = shuffle(all_captions,
                                          all_img_name_vector,
                                          random_state=1)

# Select the first 30000 captions from the shuffled set
num_examples = 10000
train_captions = train_captions[:num_examples]
train_img_name_vector = img_name_vector[:num_examples]

In [0]:
len(train_captions), len(train_img_name_vector)

In [0]:
#create a dic of image path & captions
# trainImgCaption = dict()
# for i, j in zip(train_img_name_vector, train_captions):
#   trainImgCaption[i] = j

In [0]:
trainImgCaption = dict(zip(train_img_name_vector, train_captions))

## Save to Drive

In [0]:
from pickle import dump, load

# # Save the bottleneck train features to disk
with open(DATASET_PATH + "/Pickle/coco_trainImgCaption_10K.pkl", "wb") as encoded_pickle:
    dump(trainImgCaption, encoded_pickle)

#load dic
# trainImgCaption = load(open(DATASET_PATH + "/Pickle/coco_trainImgCaption_10K.pkl", "rb"))

In [0]:
ttt = load(open(DATASET_PATH + "/Pickle/coco_trainImgCaption_10K.pkl", "rb"))

In [0]:
len(ttt)

In [0]:
i = list(trainImgCaption.keys())

# {'/content/train2014/COCO_train2014_000000324909.jpg': 
# '<start> A skateboarder performing a trick on a skateboard ramp. <end>',


In [0]:
len(train_img_name_vector), len(i)

In [0]:
j = list(trainImgCaption.values())[:5]

In [0]:
display_img('/content/train2014/COCO_train2014_000000324909.jpg')

In [0]:
trainImgCaption

In [0]:
import cv2
count = 1

img_path_on_drive = []
caption_path_on_drive = []

for key, caption in trainImgCaption.items():

  #read image from cache
  image = cv2.imread(key)

  #return image_name = COCO_train2014_000000432395.jpg
  image_name = key[len('/content/train2014/'):]

  #write to Google Drive for offline under coco_2 folder
  #COCO_dataset_2 = '/content/drive/My Drive/_AI/Image Captioning/coco_2/'
  cv2.imwrite(COCO_dataset_2 + image_name, image)

  img_path_on_drive.append(COCO_dataset_2 + image_name)
  caption_path_on_drive.append(caption)
  
  # print(count)
  # count = count + 1

  # # if count > 1:
  # #   break

trainImgCaption_on_drive = dict(zip(img_path_on_drive, caption_path_on_drive))

In [0]:
len(trainImgCaption_on_drive)

In [0]:
from pickle import dump, load

# # Save the bottleneck train features to disk
with open(DATASET_PATH + "/Pickle/coco_trainImgCaption_on_drive_10K.pkl", "wb") as encoded_pickle:
    dump(trainImgCaption_on_drive, encoded_pickle)

#load dic
# trainImgCaption_on_drive = load(open(DATASET_PATH + "/Pickle/coco_trainImgCaption_on_drive_10K.pkl", "rb"))

In [0]:
#Testing saved files
ttt = load(open(DATASET_PATH + "/Pickle/coco_trainImgCaption_on_drive_10K.pkl", "rb"))
ttt

In [0]:
count = 1
for i, j in trainImgCaption_on_drive.items():
  print(j)
  display_img(i)  
  count = count + 1

  if count > 2:
    break
  

In [0]:
display_img(i)

In [0]:
txt =  '/content/train2014/COCO_train2014_000000027074.jpg'

In [0]:
a = txt[len('/content/train2014/'):]
COCO_dataset + a

In [0]:
import glob

# Below path contains all the images
# images = '../../storage/image_caption/dataset/Flicker8k_Dataset/'
# images = dataset_img_path + '/'
# Create a list of all image names in the directory
coco_img = glob.glob(COCO_dataset + '*.jpg')
len(coco_img)

In [0]:
coco_img[1]

In [0]:
train_img_name_vector[1]

In [0]:
img_path = train_img_name_vector[1]

# img_path = coco_img[10]

import matplotlib.pyplot as plt
def display_img(img_path):
  im = plt.imread(img_path)
  plt.imshow(im)

display_img(img_path)

In [0]:
img = '/content/drive/My Drive/_AI/Image Captioning/coco/COCO_train2014_000000427805.jpg'

img_id = img[len('/content/drive/My Drive/_AI/Image Captioning/coco/COCO_train2014_'):len('/content/drive/My Drive/_AI/Image Captioning/coco/COCO_train2014_000000427805')]

img_id_number = img_id.lstrip('0')

In [0]:
start = len('/content/drive/My Drive/_AI/Image Captioning/coco/COCO_train2014_')
end = len('/content/drive/My Drive/_AI/Image Captioning/coco/COCO_train2014_000000427805')
def get_image_id(full_img_path):
  img_id = full_img_path[start:end]
  img_id_number = img_id.lstrip('0')
  return img_id_number

In [0]:
get_image_id(img)

In [0]:
coco_img_id = []

for i in coco_img:
  coco_img_id.append(get_image_id(i))

In [0]:
coco_img_id

In [0]:
COCO_dataset + 'COCO_train2014_'

In [0]:
# # Read the json file
# with open(annotation_file, 'r') as f:
#     annotations = json.load(f)

# # Store captions and image names in vectors
# train_captions = []
# train_img_name_vector = []

# # for annot in annotations['annotations']:    

# for i in coco_img_id:
#   if i in annotations['annotations']['image_id']

#     if annot['image_id'] in coco_img_id:
#       caption = '<start> ' + annot['caption'] + ' <end>'
#       full_coco_image_path = COCO_dataset + 'COCO_train2014_' + '%012d.jpg' % (annot['image_id'])
#       train_img_name_vector.append(full_coco_image_path)
#       train_captions.append(caption)

# # Shuffle captions and image_names together
# # Set a random state
# # train_captions, img_name_vector = shuffle(all_captions,
# #                                           all_img_name_vector,
# #                                           random_state=1)

# # Select the first 30000 captions from the shuffled set
# # num_examples = 10000
# # train_captions = train_captions[:num_examples]
# # train_img_name_vector = img_name_vector[:num_examples]

## Load from Google drive

In [0]:
from pickle import dump, load

#load dic
trainImgCaption_on_drive = load(open(DATASET_PATH + "/Pickle/coco_trainImgCaption_on_drive_10K.pkl", "rb"))

In [0]:
train_img_name_vector = list(trainImgCaption_on_drive.keys())
train_captions = list(trainImgCaption_on_drive.values())

len(train_img_name_vector), len(train_captions)

In [0]:
#Test
import matplotlib.pyplot as plt
def display_img(img_path):
  im = plt.imread(img_path)
  plt.imshow(im)

count = 1
for i, j in zip(train_img_name_vector, train_captions):
  print(j)
  display_img(i)  
  count = count + 1

  if count > 1:
    break

## Preprocess the images using InceptionV3
Next, you will use InceptionV3 (which is pretrained on Imagenet) to classify each image. You will extract features from the last convolutional layer.

First, you will convert the images into InceptionV3's expected format by:
* Resizing the image to 299px by 299px
* [Preprocess the images](https://cloud.google.com/tpu/docs/inception-v3-advanced#preprocessing_stage) using the [preprocess_input](https://www.tensorflow.org/api_docs/python/tf/keras/applications/inception_v3/preprocess_input) method to normalize the image so that it contains pixels in the range of -1 to 1, which matches the format of the images used to train InceptionV3.

In [0]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

## Initialize InceptionV3 and load the pretrained Imagenet weights

Now you'll create a tf.keras model where the output layer is the last convolutional layer in the InceptionV3 architecture. The shape of the output of this layer is ```8x8x2048```. You use the last convolutional layer because you are using attention in this example. You don't perform this initialization during training because it could become a bottleneck.

* You forward each image through the network and store the resulting vector in a dictionary (image_name --> feature_vector).
* After all the images are passed through the network, you pickle the dictionary and save it to disk.




In [0]:
image_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

## Caching the features extracted from InceptionV3

You will pre-process each image with InceptionV3 and cache the output to disk. Caching the output in RAM would be faster but also memory intensive, requiring 8 \* 8 \* 2048 floats per image. At the time of writing, this exceeds the memory limitations of Colab (currently 12GB of memory).

Performance could be improved with a more sophisticated caching strategy (for example, by sharding the images to reduce random access disk I/O), but that would require more code.

The caching will take about 10 minutes to run in Colab with a GPU. If you'd like to see a progress bar, you can: 

1. install [tqdm](https://github.com/tqdm/tqdm):

    `!pip install tqdm`

2. Import tqdm:

    `from tqdm import tqdm`

3. Change the following line:

    `for img, path in image_dataset:`

    to:

    `for img, path in tqdm(image_dataset):`


In [0]:
# Get unique images
# encode_train = sorted(set(img_name_vector))

encode_train = sorted(set(train_img_name_vector))

# Feel free to change batch_size according to your system configuration
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(
  load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)

for img, path in image_dataset:
  batch_features = image_features_extract_model(img)
  batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))

  for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    np.save(path_of_feature, bf.numpy())

In [0]:
image_dataset

In [0]:
path

In [0]:
path_of_feature

In [0]:
bf.numpy()

## Preprocess and tokenize the captions

* First, you'll tokenize the captions (for example, by splitting on spaces). This gives us a  vocabulary of all of the unique words in the data (for example, "surfing", "football", and so on).
* Next, you'll limit the vocabulary size to the top 5,000 words (to save memory). You'll replace all other words with the token "UNK" (unknown).
* You then create word-to-index and index-to-word mappings.
* Finally, you pad all sequences to be the same length as the longest one.

In [0]:
# Find the maximum length of any caption in our dataset
def calc_max_length(tensor):
    return max(len(t) for t in tensor)

In [0]:
# Choose the top 5000 words from the vocabulary
top_k = 5000
# tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
#                                                   oov_token="<unk>",
#                                                   filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')

tokenizer.fit_on_texts(train_captions)
train_seqs = tokenizer.texts_to_sequences(train_captions)

In [0]:
len(train_seqs)
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'
train_seqs = tokenizer.texts_to_sequences(train_captions)

cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')
max_length = calc_max_length(train_seqs)
max_length

tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'

# Create the tokenized vectors
train_seqs = tokenizer.texts_to_sequences(train_captions)

# Calculates the max_length, which is used to store the attention weights
max_length = calc_max_length(train_seqs)
max_length

In [0]:
# tokenizer.word_index['<pad>'] = 0
# tokenizer.index_word[0] = '<pad>'

In [0]:
# Create the tokenized vectors
# train_seqs = tokenizer.texts_to_sequences(train_captions)

In [0]:
train_seqs

In [0]:
# Pad each vector to the max_length of the captions
# If you do not provide a max_length value, pad_sequences calculates it automatically
# cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

In [0]:
# # Calculates the max_length, which is used to store the attention weights
# max_length = calc_max_length(train_seqs)
# max_length

In [0]:
# dump(tokenizer, open('tokenizer.pkl', 'wb'))

## Split the data into training and testing

In [0]:
# Create training and validation sets using an 90-10 split
img_name_train, img_name_val, cap_train, cap_val = train_test_split(train_img_name_vector,
                                                                    cap_vector,
                                                                    test_size=0.1,
                                                                    random_state=0)

In [0]:
len(img_name_train), len(cap_train), len(img_name_val), len(cap_val)

## Create a tf.data dataset for training



 Our images and captions are ready! Next, let's create a tf.data dataset to use for training our model.

In [0]:
# Feel free to change these parameters according to your system's configuration

BATCH_SIZE = 64
BUFFER_SIZE = 1000
embedding_dim = 256
units = 512
top_k = 5000
vocab_size = top_k + 1
num_steps = len(img_name_train) // BATCH_SIZE
# Shape of the vector extracted from InceptionV3 is (64, 2048)
# These two variables represent that vector shape
features_shape = 2048
attention_features_shape = 64

In [0]:
# Load the numpy files
def map_func(img_name, cap):
  img_tensor = np.load(img_name.decode('utf-8')+'.npy')
  return img_tensor, cap

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((img_name_train, cap_train))

# Use map to load the numpy files in parallel
dataset = dataset.map(lambda item1, item2: tf.numpy_function(
          map_func, [item1, item2], [tf.float32, tf.int32]),
          num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Shuffle and batch
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

## Model

Fun fact: the decoder below is identical to the one in the example for [Neural Machine Translation with Attention](../sequences/nmt_with_attention.ipynb).

The model architecture is inspired by the [Show, Attend and Tell](https://arxiv.org/pdf/1502.03044.pdf) paper.

* In this example, you extract the features from the lower convolutional layer of InceptionV3 giving us a vector of shape (8, 8, 2048).
* You squash that to a shape of (64, 2048).
* This vector is then passed through the CNN Encoder (which consists of a single Fully connected layer).
* The RNN (here GRU) attends over the image to predict the next word.

In [0]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, features, hidden):
    # features(CNN_encoder output) shape == (batch_size, 64, embedding_dim)

    # hidden shape == (batch_size, hidden_size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
    hidden_with_time_axis = tf.expand_dims(hidden, 1)

    # score shape == (batch_size, 64, hidden_size)
    score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))

    # attention_weights shape == (batch_size, 64, 1)
    # you get 1 at the last axis because you are applying score to self.V
    attention_weights = tf.nn.softmax(self.V(score), axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * features
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [0]:
class CNN_Encoder(tf.keras.Model):
    # Since you have already extracted the features and dumped it using pickle
    # This encoder passes those features through a Fully connected layer
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        # shape after fc == (batch_size, 64, embedding_dim)
        self.fc = tf.keras.layers.Dense(embedding_dim)

    def call(self, x):
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x

In [0]:
class RNN_Decoder(tf.keras.Model):
  def __init__(self, embedding_dim, units, vocab_size):
    super(RNN_Decoder, self).__init__()
    self.units = units

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc1 = tf.keras.layers.Dense(self.units)
    self.fc2 = tf.keras.layers.Dense(vocab_size)

    self.attention = BahdanauAttention(self.units)

  def call(self, x, features, hidden):
    # defining attention as a separate model
    context_vector, attention_weights = self.attention(features, hidden)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # shape == (batch_size, max_length, hidden_size)
    x = self.fc1(output)

    # x shape == (batch_size * max_length, hidden_size)
    x = tf.reshape(x, (-1, x.shape[2]))

    # output shape == (batch_size * max_length, vocab)
    x = self.fc2(x)

    return x, state, attention_weights

  def reset_state(self, batch_size):
    return tf.zeros((batch_size, self.units))

In [0]:
embedding_dim = 256
units = 512
top_k = 5000
vocab_size = top_k + 1

encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Checkpoint

In [0]:
checkpoint_path = "./checkpoints/train"
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer = optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

In [0]:
start_epoch = 0
if ckpt_manager.latest_checkpoint:
  start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])

In [0]:
!ls "./checkpoints/train"

## Training

* You extract the features stored in the respective `.npy` files and then pass those features through the encoder.
* The encoder output, hidden state(initialized to 0) and the decoder input (which is the start token) is passed to the decoder.
* The decoder returns the predictions and the decoder hidden state.
* The decoder hidden state is then passed back into the model and the predictions are used to calculate the loss.
* Use teacher forcing to decide the next input to the decoder.
* Teacher forcing is the technique where the target word is passed as the next input to the decoder.
* The final step is to calculate the gradients and apply it to the optimizer and backpropagate.


In [0]:
# adding this in a separate cell because if you run the training cell
# many times, the loss_plot array will be reset
loss_plot = []

In [0]:
@tf.function
def train_step(img_tensor, target):
  loss = 0

  # initializing the hidden state for each batch
  # because the captions are not related from image to image
  hidden = decoder.reset_state(batch_size=target.shape[0])

  dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)

  with tf.GradientTape() as tape:
      features = encoder(img_tensor)

      for i in range(1, target.shape[1]):
          # passing the features through the decoder
          predictions, hidden, _ = decoder(dec_input, features, hidden)

          loss += loss_function(target[:, i], predictions)

          # using teacher forcing
          dec_input = tf.expand_dims(target[:, i], 1)

  total_loss = (loss / int(target.shape[1]))

  trainable_variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, trainable_variables)

  optimizer.apply_gradients(zip(gradients, trainable_variables))

  return loss, total_loss

In [0]:
EPOCHS = 10

for epoch in range(start_epoch, EPOCHS):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(dataset):
        batch_loss, t_loss = train_step(img_tensor, target)
        total_loss += t_loss

        if batch % 100 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(
              epoch + 1, batch, batch_loss.numpy() / int(target.shape[1])))
    # storing the epoch end loss value to plot later
    loss_plot.append(total_loss / num_steps)

    if epoch % 5 == 0:
      ckpt_manager.save()

    print ('Epoch {} Loss {:.6f}'.format(epoch + 1,
                                         total_loss/num_steps))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [0]:
plt.plot(loss_plot)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Plot')
plt.show()

## Caption!

* The evaluate function is similar to the training loop, except you don't use teacher forcing here. The input to the decoder at each time step is its previous predictions along with the hidden state and the encoder output.
* Stop predicting when the model predicts the end token.
* And store the attention weights for every time step.

In [0]:
def evaluate(image):
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = decoder.reset_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)

        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

        predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == '<end>':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot

In [0]:
def plot_attention(image, result, attention_plot):
    temp_image = np.array(Image.open(image))

    fig = plt.figure(figsize=(15, 10))

    len_result = len(result)
    for l in range(len_result):
        temp_att = np.resize(attention_plot[l], (10, 10))
        ax = fig.add_subplot(len_result//2, len_result//2, l+1)
        ax.set_title(result[l])
        img = ax.imshow(temp_image)
        ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

    plt.tight_layout()
    plt.show()

In [0]:
attention_plot

In [0]:
# captions on the validation set
rid = np.random.randint(0, len(img_name_val))
# rid = 157
image = img_name_val[rid]
real_caption = ' '.join([tokenizer.index_word[i] for i in cap_val[rid] if i not in [0]])
result, attention_plot = evaluate(image)

print ('Real Caption:', real_caption)
print ('Prediction Caption: <start>', ' '.join(result))
plot_attention(image, result, attention_plot)

# opening the image
print('#ID: ', rid)
Image.open(image)

## Try it on your own images
For fun, below we've provided a method you can use to caption your own images with the model we've just trained. Keep in mind, it was trained on a relatively small amount of data, and your images may be different from the training data (so be prepared for weird results!)


In [0]:
image_url = 'https://tensorflow.org/images/surf.jpg'
# image_url =  'https://vtv1.mediacdn.vn/thumb_w/650/2019/10/18/bong-da-nu-01-2304-15714114665531563395741.jpg'
image_extension = image_url[-4:]
image_path = tf.keras.utils.get_file('image'+image_extension, origin=image_url)

result, attention_plot = evaluate(image_path)
print ('Prediction Caption:', ' '.join(result))
plot_attention(image_path, result, attention_plot)
# opening the image
Image.open(image_path)

In [0]:
image_extension

In [0]:
image_path

In [0]:
image

In [0]:
# image_url =  'https://vtv1.mediacdn.vn/thumb_w/650/2019/10/18/bong-da-nu-01-2304-15714114665531563395741.jpg'
# image_url = 'https://cdn.thethao247.vn/upload/vanhung/2019/12/10/truc-tiep-u22-viet-nam-vs-u22-indonesia-chung-ket-sea-games-30-20191575984470.jpg'
image_url = 'https://vnn.onecmscdn.com/2019/12/06/photo-1-15761567061301811764033.jpg'
# image_url = 'https://image.thanhnien.vn/660/uploaded/quocviet/2020_01_02/ds/tuyen-u23vietnam-tap-buoi-dau-tai-bangkok-huu_5201_vhxc.jpg'
# image_extension = image_url[-4:]
# image_path = tf.keras.utils.get_file('image'+image_extension, origin=image_url)

image_path = tf.keras.utils.get_file('image3.jpg', origin=image_url)

result, attention_plot = evaluate(image_path)
print ('Prediction Caption:', ' '.join(result))
plot_attention(image_path, result, attention_plot)
# opening the image
Image.open(image_path)

# Next steps

Congrats! You've just trained an image captioning model with attention. Next, take a look at this example [Neural Machine Translation with Attention](../sequences/nmt_with_attention.ipynb). It uses a similar architecture to translate between Spanish and English sentences. You can also experiment with training the code in this notebook on a different dataset.

In [0]:
len(decoder.weights)

In [0]:
from pickle import dump, load

# # Save the bottleneck train features to disk
with open(DATASET_PATH + "/Pickle/COCO_decoder_100.pkl", "wb") as encoded_pickle:
    dump(decoder, encoded_pickle)

# with open(DATASET_PATH + "/Pickle/COCO_encoder_100.pkl", "wb") as encoded_pickle:
#     dump(encoder, encoded_pickle)

In [0]:
with open(DATASET_PATH + "/Pickle/COCO_tokenizer_100.pkl", "wb") as encoded_pickle:
    dump(tokenizer, encoded_pickle)

# Connect and load data from Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
ROOT_DIR = '/content/drive/My Drive/_AI/Image Captioning'

models_path = ROOT_DIR + '/models'


In [0]:
models_path

In [0]:
# save entire model
# decoder.save(models_path + '/image_captioning_attention_entire_model_decoder_v1.tf')

In [0]:
decoder.save_weights(models_path + '/image_captioning_attention_decoder_v1.h5')

In [0]:
encoder.save_weights(models_path + '/image_captioning_attention_encoder_v1.h5')

In [0]:
decoder.load_weights(models_path + '/image_captioning_attention_decoder_v1.h5')

In [0]:
decoder.summary()

In [0]:
encoder.load_weights(models_path + '/image_captioning_attention_encoder_v1.h5')

#Public to Web

## Define web templates

In [0]:
#ROOT_DIR = '/content/drive/My Drive/_AI/Image Captioning'
web_dir = ROOT_DIR + '/web_coco/'
templates_dir = web_dir + 'templates/'
static_dir = web_dir + 'static/'

In [0]:
with open(templates_dir + 'predict_form.html', 'wt') as f:
    f.write('''
    <style>
    table, td, th {  
        border: 1px solid #ddd;
        text-align: left;
    }

    table {
        border-collapse: collapse;
        width: 100%;
    }

    th, td {
        padding: 15px;
    }
    </style>
</head>
<body>

<h1>Image Captioning Demo 2 - Show, Attend and Tell Approach </h1>

<table>
  <tr>
    <th width="40%">Load local image </th>
    <th>Load image from URL</th>
  </tr>
  <tr>
    <td>
    <form action = "/predict_upload" method = "POST" enctype = "multipart/form-data">
        <input type = "file" name = "file" />
        <input type = "submit"/>
    </form>
    </td>
    <td>
        <form action = "/predict_url" method = "POST">
            <input name="imageurl" style="width: 600px">
            <input type = "submit"/>
        </form>
    </td>
  </tr>
</table>
    
        ''')

In [0]:
with open(templates_dir + 'predict_result.html', 'wt') as f:
    f.write('''
<html>
    <body>
      <div>
        <h2>Input Image</h2>
        <img src="{{ url_for('static', filename = 'predict.jpg') }}" width="600"/>
      </div>
      <div>
        <h2>Prediction Caption: {{txt_caption}}</h2>
        <img src="{{ url_for('static', filename = 'attention_plot.jpg') }}" width="100%"/>
      </div>
    </body>
</html>
''')

##App NGROK

In [0]:
!pip install flask-ngrok

In [0]:
def plot_attention_4web(image, result, attention_plot):
    temp_image = np.array(Image.open(image))

    fig = plt.figure(figsize=(15, 10))

    len_result = len(result)
    for l in range(len_result):
        temp_att = np.resize(attention_plot[l], (10, 10))
        ax = fig.add_subplot(len_result//2, len_result//2, l+1)
        ax.set_title(result[l])
        img = ax.imshow(temp_image)
        ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

    plt.tight_layout()
    plt.show()

    plt.savefig(static_dir + 'attention_plot.jpg')
    fig.savefig(static_dir + 'attention_plot.jpg')

In [0]:
import urllib
import cv2
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request
import base64
import re
import sys
import os

app = Flask(__name__, static_url_path='', template_folder=templates_dir, static_folder=static_dir)
run_with_ngrok(app)

# decoding an image from base64 into raw representation
def convertImage(imgData1):
    imgstr = re.search(r'base64,(.*)', str(imgData1)).group(1)
    with open('output.png', 'wb') as output:
        output.write(base64.b64decode(imgstr))

def predict(np_img):
    imgData = request.get_data()
    # os.remove(static_dir + 'predict.jpg')

    image_path = static_dir + 'predict.jpg'

    cv2.imwrite(image_path, np_img)
    x = cv2.imread(image_path)

    result, attention_plot = evaluate(image_path)

    pre_caption = ' '.join(result)

    print ('Prediction Caption:', pre_caption)

    plot_attention_4web(image_path, result, attention_plot)
    # opening the image
    # Image.open(image_path)

    # pre_caption = predict_caption(static_dir + 'predict.jpg')    
     
    return render_template('predict_result.html', txt_caption = pre_caption)

#START WEB APP
@app.route('/predict', methods=['GET', 'POST'])
def predict_form():
    return render_template('predict_form.html') 
     
    
@app.route('/predict_upload', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        f = request.files['file']
        img = f.read()
        np_arr = np.fromstring(img, np.uint8)
        np_img = cv2.imdecode(np_arr, cv2.IMREAD_UNCHANGED)  
        return predict(np_img)
        

@app.route('/predict_url', methods=['GET', 'POST'])
def predicturl():    
    if request.method == 'POST':
        imageurl = request.form["imageurl"]
    elif request.method == 'GET':
        imageurl = request.args.get("imageurl")
    
    url = urllib.request.urlopen(imageurl)
    np_arr = np.asarray(bytearray(url.read()), dtype=np.uint8)
    np_img = cv2.imdecode(np_arr, -1)  # 'Load it as it is'
    return predict(np_img)

  #starts ngrok when the app is run
@app.route("/")
def home():
  return "<h1>Welcome to Deep Learning Demo!</h1>"
  # return render_template('index.html')

app.run()

#if __name__ == '__main__':
#    app.run(host='0.0.0.0', port='9102')     